In [1]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

## gogo

In [ ]:
tokenizer_name = 'snunlp/KR-Medium'
model_name = 'snunlp/KR-Medium'

In [ ]:
import json
import random

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [ ]:
import re
def remove_post(text):
        ''' 불필요한 기호 제거 '''
        text = text.strip()
        text = re.sub("'", "", text)
        text = re.sub('"', "", text)
        text = re.sub('《', "", text)
        text = re.sub('》', "", text)
        text = re.sub('<', "", text)
        text = re.sub('>', "", text)
        text = re.sub('〈', "", text)
        text = re.sub('〉', "", text)
        text = re.sub("\(", "", text)
        text = re.sub("\)", "", text)
        text = re.sub("‘", "", text)
        text = re.sub("’", "", text)
        text = re.sub("  ", " ", text)
        text = re.sub("#", "", text)
        return text

In [ ]:
'''
idea  
틀린 답을 바로 빈칸으로 내보내는게 과연 이득일까?
logit 확률값을 적용해서 틀린답이면 다음으로 높은 확률값이 정답일수도 있지않을까?
top 5의 확률값을 다 적용해도 틀린 답이라면 그때 빈칸을 내보내도 괜찮을듯.
'''
# start, end logit의 확률값을 이용한 예측 정답값
# logit의 상위 5개 확률을 리스트로 뽑아 틀린 정답이었다면 다음 확률로 넘어가서 확인.

def to_list(tensor):
    return tensor.detach().cpu().tolist()

def logits_change(input_ids, STA_logits, END_logits):

    # 로짓의 확률값 ~ 상위 5개를 선택 
    # 틀린 추론이었다면 다음 선택 (틀린 추론 : start > end, 길이가 너무 긴 문장.)
    change_logit = 0
    cnt = 0
    
    # 기존 정답
    save_s = STA_logits
    save_e = END_logits

    STK_start_index, STK_end_index = save_s.argmax(dim=-1), save_e.argmax(dim=-1)
    save_pred_ids = tokenizer.decode(input_ids[0][STK_start_index: STK_end_index + 1])
    #print(save_pred_ids) 

    # 바뀐 정답
    # Tensor형태의 logit의 확률값을 리스트로 만들어줌 
    STA_logits = to_list(STA_logits)[0]
    END_logits = to_list(END_logits)[0]

    # 리스트로 만든 확률값 -> 큰 순서대로 정렬 + 인덱스 
    start_idx_and_logit = sorted(enumerate(STA_logits), key=lambda x: x[1], reverse=True)
    end_idx_and_logit = sorted(enumerate(END_logits), key=lambda x: x[1], reverse=True)

    # 확률값 큰 순서대로 Top 5 
    start_idx_and_logit = start_idx_and_logit[:5]
    end_idx_and_logit = end_idx_and_logit[:5]

    TK_start_index, TK_end_index = start_idx_and_logit, end_idx_and_logit

    # 확률이 높은 순서대로 점검. 틀린 값이면 다음 확률로 넘어가고 맞는 답이면 저장후 반복문 종료
    for i in range(5):
        if TK_start_index[i][0] > TK_end_index[i][0] or TK_end_index[i][0] - TK_start_index[i][0] > 10 : 
            cnt += 1
            continue
        else : 
            change_logit += 1
            pred_ids = input_ids[0][TK_start_index[i][0]: TK_end_index[i][0] + 1]
            pred_ids = tokenizer.decode(pred_ids)
            #print(pred_ids, 'change')
            break

    if change_logit == 0 :
        return save_pred_ids
    elif cnt == 5:
        return ''
    else : 
        if pred_ids == save_pred_ids:
            #print('same answer')
            return pred_ids
        else :
            #print('different answer')
            return pred_ids

    

# start_logits , end_logits
# index를 추적하면서 시작, 종료 index에 대한 확률이 가장 높은것을 선택하는 방법.
# 만약에 차이가 큰 start, end 값을 반환할때 이 정보들을 저장하지 않고 넘긴다면? 


In [ ]:
# pred_answers에서 도출한 정답 비교
import csv

def calculate_Leven(source, ref, result_file):
    with open(source, 'r') as input1:
        with open(ref, 'r') as input2:
            with open(result_file, 'w') as csvoutput:
                reader1 = csv.reader(input1)
                reader2 = list(csv.reader(input2))
                writer = csv.writer(csvoutput)
                result = []
                mean = []
                headers = next(reader1)
                result.append(headers)
                index = 0
                for row1 in reader1:
                    #print("First row")
                    #print(row1[1])
                    index+=1
                    #print(reader2[index][1])
                    a = levenshtein_distance(row1[1], reader2[index][1])
                    print(row1[1],'////',reader2[index][1])
                    '''
                    max = 0
                    while max < 1:
                        for row2 in reader2:
                            a = distance(row1[1],row2[1])
                            print(a)
                            b = 1 - a/len(row1[1])
                            if b > max:
                                max = b
                                SKU = row2[1]
                    '''
                    mean.append(a)
                    row1.append(a)
                    result.append(row1)
                mean1 = sum(mean) / len(mean)
                print(mean1)
                writer.writerows(result)


In [ ]:
calculate_Leven('klue.csv', 'sangrimlee_bert-base-multilingual-cased-korquad_validation_pred_truncation.csv', 'result_file.csv')

In [ ]:
# logit 값에 대한 soft voting - 3 model

def to_list(tensor):
    return tensor.detach().cpu().tolist()

def logits_voting(A_model, B_model, C_model):
    # 각 모델마다 input_ids, start_logits, end_logits값 저장. 순서대로 저장 리스트에 + guids // context 필요.
    # 모델 logit 확률 top 5 구해서 모델 1 top1 모델 2 top1 모델3 top1 모델1 top2 순으로 체크.
    # print(list_dict_all[0]["start_logits"]) pickle
    # A_model

    ind = 0
    RESULT = []

    if len(A_model)!= len(B_model) or len(A_model) != len(C_model) or len(B_model) != len(C_model):
        print('error')
    else:
        llogit = len(A_model)

    while 1:
        # input_ids
        A_input_ids = A_model[ind]['input_ids']
        B_input_ids = B_model[ind]['input_ids']
        C_input_ids = C_model[ind]['input_ids']

        # guid
        A_guid = A_model[ind]['guid']
        B_guid = A_model[ind]['guid']
        C_guid = A_model[ind]['guid']

        if A_input_ids != B_input_ids or B_input_ids != C_input_ids or A_input_ids != C_input_ids:
            print('input_error')
        if A_guid != B_guid or B_guid != C_guid or A_guid != C_guid:
            print('guid_error')

        # logit 값 순서대로 불러오기기
        A_start_logit = A_model[ind]['start_logits']
        A_end_logit = A_model[ind]['end_logits']
        B_start_logit = B_model[ind]['start_logits']
        B_end_logit = B_model[ind]['end_logits']
        C_start_logit = C_model[ind]['start_logits']
        C_end_logit = C_model[ind]['end_logits']

        # 로짓값 -> 확률 큰 순서대로 리스트형태 + 인덱스 -> top 5 자르기기
        A_start_idx_and_logit = (sorted(enumerate(A_start_logit), key=lambda x: x[1], reverse=True))[:5]
        A_end_idx_and_logit = (sorted(enumerate(A_end_logit), key=lambda x: x[1], reverse=True))[:5]
        B_start_idx_and_logit = (sorted(enumerate(B_start_logit), key=lambda x: x[1], reverse=True))[:5]
        B_end_idx_and_logit = (sorted(enumerate(B_end_logit), key=lambda x: x[1], reverse=True))[:5]
        C_start_idx_and_logit = (sorted(enumerate(C_start_logit), key=lambda x: x[1], reverse=True))[:5]
        C_end_idx_and_logit = (sorted(enumerate(C_end_logit), key=lambda x: x[1], reverse=True))[:5]


        cnt = 0
        for i in range(5):
            # 모델 A, B, C 에 대한 각각의 로짓값 저장해서 확률 5 리스트로 저장 후 비교.
            # A->B->C->A 순? A2 > B1 ? 어떤 순서대로 비교할건지 
            # 정답인 경우 멈추고 값 저장 후 종료
            # 토크나이저 다 다른데 어떻게 비교함? -> 정답 context 
            # soft voting -> start + end 확률값 더해서 순위대로 정렬 후 정답인지 확인 -> 정답일때는 해당 모델에 대한 input 불러와서 answer, guid 저장.

            if A_start_idx_and_logit[i][0] < A_end_idx_and_logit[i][0] or A_end_idx_and_logit[i][0] - A_start_idx_and_logit[i][0] <= 10 : 
                cnt += 1
                pred_ids = A_input_ids[A_start_idx_and_logit[i][0]: A_end_idx_and_logit[i][0] + 1] # 해당 정답에 대한 인덱스 ~> context 변환 하는 과정 필요.
                # 토큰 -> 정답 변경.
                A_token = A_model['tokenizer_name'] # 저장형태 확인 // 근데 이렇게하면 decode가 되나? 해봐야 될듯?
                tokenizer = AutoTokenizer.from_pretrained(A_token)
                pred_ids = tokenizer.decode(pred_ids)
                guid = A_guid
                break

            elif B_start_idx_and_logit[i][0] < B_end_idx_and_logit[i][0] or B_end_idx_and_logit[i][0] - B_start_idx_and_logit[i][0] <= 10 : 
                cnt += 1
                pred_ids = B_input_ids[B_start_idx_and_logit[i][0]: B_end_idx_and_logit[i][0] + 1]
                B_token = B_model['tokenizer_name']
                tokenizer = AutoTokenizer.from_pretrained(B_token)
                pred_ids = tokenizer.decode(pred_ids)
                guid = B_guid
                break

            elif C_start_idx_and_logit[i][0] < C_end_idx_and_logit[i][0] or C_end_idx_and_logit[i][0] - C_start_idx_and_logit[i][0] <= 10 : 
                cnt += 1
                pred_ids = C_input_ids[C_start_idx_and_logit[i][0]: C_end_idx_and_logit[i][0] + 1]
                C_token = C_model['tokenizer_name']
                tokenizer = AutoTokenizer.from_pretrained(C_token)
                pred_ids = tokenizer.decode(pred_ids)
                guid = C_guid
                break


                 

        # guids??? 저장해야함.
        if cnt == 0:
           tp = (guid,'')
           RESULT.append(tp)
        else:
           tp = (guid,pred_ids)
           RESULT.append(tp)

        if llogit == ids:
            break
        ids += 1

    return RESULT
            # 1 23 맞는 정답만 도출.




In [ ]:
### downloading pickle ###
import pickle

with open('input_ids_and_logits.pickle1111', 'rb') as handle:
    pickle1 = pickle.load(handle)

with open('input_ids_and_logits.pickle2222', 'rb') as handle:
    pickle2 = pickle.load(handle)

with open('input_ids_and_logits.pickle3333', 'rb') as handle:
    pickle3 = pickle.load(handle)
    

In [1]:
from google.colab import files
uploaded = files.upload()

Saving input_ids_and_logits.pickle to input_ids_and_logits.pickle


In [ ]:
pred_answers = logits_voting(pickle1,pickle2,pickle3)

In [ ]:
print(pred_answers[:10])

In [ ]:


import csv
f = open('voting1.csv','w', newline='')
wr = csv.writer(f)
wr.writerow(['Id','Predicted'])

for tp in pred_answers:
    wr.writerow([tp[0],tp[1]])

f.close()

In [ ]:
#@title 레벤슈타인거리 함수
# pred_answers에서 도출한 정답 비교
import csv

def calculate_Leven(source, ref, result_file):
    with open(source, 'r') as input1:
        with open(ref, 'r') as input2:
            with open(result_file, 'w') as csvoutput:
                reader1 = csv.reader(input1)
                reader2 = list(csv.reader(input2))
                writer = csv.writer(csvoutput)
                result = []
                mean = []
                headers = next(reader1)
                result.append(headers)
                index = 0
                for row1 in reader1:
                    #print("First row")
                    #print(row1[1])
                    index+=1
                    #print(reader2[index][1])
                    a = levenshtein_distance(row1[1], reader2[index][1])
                    #print(row1[1],'////',reader2[index][1])
                    '''
                    max = 0
                    while max < 1:
                        for row2 in reader2:
                            a = distance(row1[1],row2[1])
                            print(a)
                            b = 1 - a/len(row1[1])
                            if b > max:
                                max = b
                                SKU = row2[1]
                    '''
                    mean.append(a)
                    row1.append(a)
                    result.append(row1)
                mean1 = sum(mean) / len(mean)
                print(mean1)
                writer.writerows(result)

# Levenshtein_distance (Evaluation)

import numpy 
import torch
import os

def levenshtein_distance(s1,s2, debug=False): #레벤슈타인 거리 eval / 정답 s1과 도출한 모델 s2 비교 평가
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1] # levenshtein_distance 값 출력력


def LD_SCORE(start_positions, end_positions, input_ids, STA_logits, END_logits):

    answer1 = [] # 정답 저장
    answer2 = [] # 예측 정답 저장

    if len(input_ids) != BATCH_SIZE: #오류 해결
       print("input_ids ERROR")
       return 0

    for i in range(BATCH_SIZE): # 기존 정답 // index 1 is out of bounds for dimension 0 with size 1 오류 발생
        if input_ids[i] == [] :
            print("input_ids ERROR")
            continue       
        else:
            PRED_IDE = input_ids[i][start_positions[i]: end_positions[i] + 1]
            PRED_ANS = tokenizer.decode(PRED_IDE)
            answer1.append(PRED_ANS)
            #print(PRED_ANS)

    for i in range(BATCH_SIZE): # 예측 정답
        if input_ids[i] == [] :
            print("input_ids ERROR")
            continue
        else:
            TK_start_index, TK_end_index = STA_logits.argmax(dim=-1), END_logits.argmax(dim=-1)
            PRED_IDE2 = input_ids[i][TK_start_index[i]: TK_end_index[i] + 1]
            PRED_ANS2 = tokenizer.decode(PRED_IDE2)
            answer2.append(PRED_ANS2)
            # print(PRED_ANS2)

    batch_score = LD_comparison(answer1, answer2)

    return batch_score


def LD_comparison(answer1,answer2):

    # 배치마다 레벤슈타인 거리 평균 구해서 출력. (train, valid 과정에서 배치마다 평균거리 구하고.)
    # train 전체 평균 거리
    # valid 전체 평균 거리

    batch_LD_score = []

    for i in range(BATCH_SIZE):
        if answer1[i] == answer2[i]: # 같으면 LD 구하는 과정 생략.
            batch_LD_score.append(0)
        else:
            batch_LD_score.append(levenshtein_distance(answer1[i],answer2[i]))

    sum_LD_score = sum(batch_LD_score)
    LD_avg = sum_LD_score / BATCH_SIZE # 배치의 레벤슈타인 거리 평균
    print(LD_avg)

    return LD_avg



'''
levenshtein_distance 값이 튀는 현상 방지.
 - 길이가 너무 긴 정답 삭제 max_length = 20
 - 정답과 어느정도 길이차이나는 값 삭제(위와 동일) 5.9 -> 2정도 차이나는 값 삭제 7.9
 

자연어처리
자연어처리과정 2
0 5

정답 길이보다 예측 정답 길이가 길지만 2배이상 차이가 나지 않는 경우에
0으로 처리하면 LD_SCORE가 더 안나옴
따라서 위 상황에서는 1.5 ~ 1.8 정도 해당하는 길이로 잘라서 반환.

test 답 X
train 과정 나온 답 -> 바꿔서 바꾼 데이터로 


** max_len = 5.9 * 2 = 12

train 2번
1 train 원래 데이터.
2 train LD 변환한 데이터로 한번.

'''



In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
calculate_Leven('ainize_klue-bert-base-mrc_LogitPRED.csv', 'monologg_koelectra-small-v2-distilled-korquad-384_Logitpred_8trunc.csv', 'result_file.csv')

In [ ]:
calculate_Leven('ainize_klue-bert-base-mrc.csv', 'monologg_koelectra-small-v2-distilled-korquad-384_Logitpred_8trunc.csv', 'result_file.csv')

In [ ]:
calculate_Leven('ainize_klue-bert-base-mrc_10truncation_real.csv', 'monologg_koelectra-small-v2-distilled-korquad-384_Logitpred_8trunc.csv', 'result_file.csv')

In [ ]:
calculate_Leven('ainize_klue-bert-base-mrc-10truncation.csv', 'monologg_koelectra-small-v2-distilled-korquad-384_Logitpred_8trunc.csv', 'result_file.csv')

In [ ]:
calculate_Leven('klue_10trunc_postprocess.csv', 'monologg_koelectra-small-v2-distilled-korquad-384_Logitpred_8trunc.csv', 'result_file.csv')